In [3]:
import numpy as np 
import pandas as pd

In [4]:
credits = pd. read_csv('credits.csv')
len(credits['id'])

45476

In [5]:
keywords = pd. read_csv('keywords.csv')
len(keywords['id'])

46419

In [6]:
links = pd. read_csv('links.csv')
len(links['movieId'])

45843

In [1]:
md= pd. read_csv('movies_metadata.csv')
len(md['id'])

" md= pd. read_csv('movies_metadata.csv')\nlen(md['id']) "

In [8]:
ratings_small = pd.read_csv('ratings_small.csv')
print(len(ratings_small['movieId'].unique()))
print(ratings_small.info())
print(ratings_small.isnull().sum())

9066
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [9]:
ratings= pd.read_csv('ratings.csv')
print(len(ratings['userId'].unique()))
print(ratings.info())
print(ratings.isnull().sum())

270896
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB
None
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [10]:
print(ratings[ratings['userId']==1].shape)
print(ratings[ratings['userId']==5829].shape)
print(ratings[ratings['userId']==9173].shape)

(27, 4)
(2955, 4)
(5, 4)


In [2]:
#print(ratings[ratings['userId']>1][ratings['userId']<3].shape)
#print(ratings[ratings['userId']==2].shape)

ratings_to_merge1 = ratings[ratings['userId']>5729][ratings['userId']<5930]
ratings_to_merge2 = ratings[ratings['userId']>9073][ratings['userId']<9274]

ratings_merged = pd.concat([ratings_small, ratings_to_merge1,ratings_to_merge2])
print(ratings_merged.info())
print(ratings_merged.shape)
print(ratings_merged['userId'].unique().shape)

In [14]:
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader

In [15]:
# Creation of the dataframe. Column names are irrelevant.

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(ratings_merged[['userId', 'movieId', 'rating']], reader=reader)


In [16]:
trainset = data.build_full_trainset()
#using SVD algo
algo = SVD()
algo.fit(trainset)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

#generate test set from unrated movies by users
testset = trainset.build_anti_testset()
predictions = algo.test(testset)  #predictions


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8856  0.8816  0.8835  0.8839  0.8942  0.8858  0.0044  
MAE (testset)     0.6812  0.6774  0.6800  0.6819  0.6880  0.6817  0.0035  
Fit time          7.19    8.21    7.13    7.26    7.02    7.36    0.43    
Test time         0.31    0.29    0.26    0.19    0.28    0.26    0.04    


In [17]:
print(len(predictions))
predictions[:5]

10665385


[Prediction(uid=1, iid=10, r_ui=3.5432585376840566, est=2.6059663126709927, details={'was_impossible': False}),
 Prediction(uid=1, iid=17, r_ui=3.5432585376840566, est=3.2030359251514637, details={'was_impossible': False}),
 Prediction(uid=1, iid=39, r_ui=3.5432585376840566, est=2.9063464905121084, details={'was_impossible': False}),
 Prediction(uid=1, iid=47, r_ui=3.5432585376840566, est=3.2339235386250076, details={'was_impossible': False}),
 Prediction(uid=1, iid=50, r_ui=3.5432585376840566, est=3.7727949039734465, details={'was_impossible': False})]

In [18]:
from collections import defaultdict
def get_top_n(predictions, n=3):
    '''for every user return N movies as a dict
    userId as keys, a tuple for corresponding values
        [(item id, rating estimation), ...].
    '''
    # retrieve predictions for each userId
    top_n = defaultdict(list)
    for uid, iid, r_ui, est, details in predictions:
        top_n[uid].append((iid, est))
    # get top N items
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=3)



In [19]:
print(ratings_merged['userId'].unique().shape)

(1071,)


In [21]:
#print(top_n[671])

rec_result=np.zeros((1071,4))  #use matrix for the results
i=0
for uid, user_ratings in top_n.items():
    rec_result[i,0]=uid
    rec_result[i,1:]=[iid for (iid, est) in user_ratings]
    i=i+1
rec_result=rec_result.astype('int')



#convert to DataFrame
rec_result=pd.DataFrame(rec_result,columns=['userId','rec1','rec2','rec3'])

#print(rec_result[:10])

#Build helper functions to link id to movie names
md= pd.read_csv('movies_metadata.csv')

links = pd.read_csv('links.csv')
#print(len(links_small))
links=links.dropna()
#print(len(links_small))
links['tmdbId'] = links['tmdbId'].astype('int')
#print(len(links_small['tmdbId']))
print(md['id'][19730])
print(len(md))
md.drop(md.index[19730], inplace=True)
md.drop(md.index[29502], inplace=True)
md.drop(md.index[35585], inplace=True)
print(len(md))
md['id']=pd.to_numeric(md['id'])
smd = md[md['id'].isin(links)]



#从idtomovie
def id2movie(idd):
    #print(idd)
    link=links[links.movieId==idd].tmdbId
    if len(link)==0:
        return ''
    a=smd[smd.id==int(link)]['title']
    if len(a)==0:
        b=md[md.id==int(links[links.movieId==idd].tmdbId)]['title']
        if len(b)==0:
            return ''
        else:
            return b.values[0]
    else:
        return a.values[0]

for i in range(1,4):
    rec_result['rec'+str(i)]=rec_result['rec'+str(i)].apply(id2movie)
print(rec_result[:5])

    

C:\Users\Lin Fan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1997-08-20
45466
45463
   userId                      rec1                       rec2  \
0       1  The Shawshank Redemption                    Memento   
1       2             The Godfather                  Gladiator   
2       3       There Will Be Blood                        Ran   
3       4           Cinema Paradiso                 Braveheart   
4       5        The Princess Bride  The City of Lost Children   

                                                rec3  
0                                          Chinatown  
1                                         Fight Club  
2                                 The Maltese Falcon  
3                                   Schindler's List  
4  Dr. Strangelove or: How I Learned to Stop Worr...  
userId                           1
rec1      The Shawshank Redemption
rec2                       Memento
rec3                     Chinatown
Name: 0, dtype: object


In [27]:
print(rec_result[rec_result['userId']==1])
print(rec_result[rec_result['userId']==5829])
print(rec_result[rec_result['userId']==9173])

   userId                      rec1     rec2       rec3
0       1  The Shawshank Redemption  Memento  Chinatown
     userId rec1             rec2                         rec3
770    5829       The Corporation  The Best Years of Our Lives
     userId                    rec1           rec2                      rec3
970    9173  The Godfather: Part II  The Godfather  The Shawshank Redemption


--------------------------------POPULARITY----------------------------------------------

In [30]:
from ast import literal_eval
import warnings; warnings.simplefilter('ignore')
import pymysql

In [3]:
#Read from Database
#Connect Database
conn = pymysql.connect(host='10.110.43.140',port= 3306,user = '###',passwd='####',db='sys') #db：库名
#Create Cursor
cur = conn.cursor()
#Read from database
md=pd.read_sql('SELECT * FROM db_movies.tb_movies;',conn)
cur.close()
conn.close()
#change column names
md.columns=['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count']